In [ ]:
import deepxde as dde

import numpy as np
import torch

'''
Based on these reference codes
[1] Lid-driven cavity problem: https://github.com/i207M/PINNacle/blob/595ab6898a30d27ac6cd44ff0a465482f8c52f5c/src/pde/ns.py
[2] Visualization: https://github.com/lululxvi/deepxde/issues/634
'''
dde.config.set_random_seed(42)

# PDE
def liddriven_pde(x, u):
    nu = 0.01
    u_vel, v_vel, _ = u[:, [0]], u[:, [1]], u[:, [2]]
    u_vel_x = dde.grad.jacobian(u, x, i=0, j=0)
    u_vel_y = dde.grad.jacobian(u, x, i=0, j=1)
    u_vel_xx = dde.grad.hessian(u, x, component=0, i=0, j=0)
    u_vel_yy = dde.grad.hessian(u, x, component=0, i=1, j=1)

    v_vel_x = dde.grad.jacobian(u, x, i=1, j=0)
    v_vel_y = dde.grad.jacobian(u, x, i=1, j=1)
    v_vel_xx = dde.grad.hessian(u, x, component=1, i=0, j=0)
    v_vel_yy = dde.grad.hessian(u, x, component=1, i=1, j=1)

    p_x = dde.grad.jacobian(u, x, i=2, j=0)
    p_y = dde.grad.jacobian(u, x, i=2, j=1)

    momentum_x = (u_vel * u_vel_x + v_vel * u_vel_y + p_x - nu * (u_vel_xx + u_vel_yy))
    momentum_y = (u_vel * v_vel_x + v_vel * v_vel_y + p_y - nu * (v_vel_xx + v_vel_yy))
    continuity = u_vel_x + v_vel_y

    return [momentum_x, momentum_y, continuity]

# Geometry
bbox=[0, 1, 0, 1]
geom = dde.geometry.Rectangle(xmin=[bbox[0], bbox[2]], xmax=[bbox[1], bbox[3]])

# BC
def boundary_top(x, on_boundary):
    return on_boundary and np.isclose(x[1], bbox[3])

def boundary_not_top(x, on_boundary):
    return on_boundary and not np.isclose(x[1], bbox[3])

bc_top_u = dde.DirichletBC(geom, (lambda _: 1), boundary_top, component=0)
bc_top_v = dde.DirichletBC(geom, (lambda _: 0), boundary_top, component=1)
bc_wall_u = dde.DirichletBC(geom, (lambda _: 0), boundary_not_top, component=0)
bc_wall_v = dde.DirichletBC(geom, (lambda _: 0), boundary_not_top, component=1)
bcs = [bc_top_u, bc_top_v, bc_wall_u, bc_wall_v]
# bcs = [bc_wall_u, bc_wall_v]

# Data
data = dde.data.PDE(geom, liddriven_pde, bcs, num_domain=2000, num_boundary=400, num_test=5000)

# Model
layer_size = [2] + [20]*5 + [3]
activation = 'tanh'
initializer = 'Glorot uniform'
net = dde.nn.FNN(layer_size, activation, initializer)

# Hard-constraints
# def output_transform(x, u):
#     '''
#     Hard-constraints are imposed only on the top plane
#     : when y=1 -> u=1 & v=0
#     '''
#     u_x = u[:, [0]] * (x[:, [1]] - 1) + 1
#     u_v = u[:, [1]] * (x[:, [1]] - 1)
#     return torch.concat((u_x, u_v, u[:,[2]]), axis=1)
#
# net.apply_output_transform(output_transform)

# Compile & Train - ADAM
'''
For more options: https://deepxde.readthedocs.io/en/latest/modules/deepxde.html#module-deepxde.model
'''
model = dde.Model(data, net)
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations = 10000, display_every = 100, model_save_path = './')
# dde.saveplot(losshistory, train_state, issave = True, isplot = True)

# Compile & Train - L-BFGS-B
model.compile(optimizer = 'L-BFGS-B')
losshistory, train_state = model.train(display_every = 100, model_save_path = './')

Compiling model...
'compile' took 0.000086 s

Training model...

Step      Train loss                                                                Test loss                                                                 Test metric
0         [4.55e-02, 1.82e-01, 1.91e-01, 1.34e+00, 1.83e-01, 1.02e-01, 1.28e-01]    [4.58e-02, 1.82e-01, 1.97e-01, 1.34e+00, 1.83e-01, 1.02e-01, 1.28e-01]    []  
100       [7.66e-04, 1.15e-04, 3.61e-03, 3.41e-02, 1.21e-04, 1.40e-01, 7.88e-05]    [6.18e-04, 1.08e-04, 3.04e-03, 3.41e-02, 1.21e-04, 1.40e-01, 7.88e-05]    []  
200       [9.24e-04, 2.51e-04, 3.41e-03, 3.18e-02, 1.39e-04, 1.33e-01, 1.69e-04]    [7.38e-04, 2.28e-04, 2.88e-03, 3.18e-02, 1.39e-04, 1.33e-01, 1.69e-04]    []  
300       [9.96e-04, 5.25e-04, 3.59e-03, 2.93e-02, 2.36e-04, 1.28e-01, 3.15e-04]    [7.83e-04, 4.68e-04, 3.03e-03, 2.93e-02, 2.36e-04, 1.28e-01, 3.15e-04]    []  
400       [9.94e-04, 7.97e-04, 3.61e-03, 2.64e-02, 3.67e-04, 1.21e-01, 5.06e-04]    [7.62e-04, 7.09e-04, 3.02e-03

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
x1_test = np.linspace(bbox[0], bbox[1], 101)
x2_test = np.linspace(bbox[2], bbox[3], 102)
X_test = np.zeros((len(x1_test)*len(x2_test), 2))
X_test[:, 0] = np.vstack((x1_test,)*len(x2_test)).reshape(-1)
X_test[:, 1] = np.vstack((x2_test,)*len(x1_test)).T.reshape(-1)
# x1_indices = np.argsort(X_test, axis=0)[:,0]
# X_test = X_test[x1_indices]
# print(X_test[:,0])
# Y_test = model.predict(X_test, operator=output_transform)
Y_test = model.predict(X_test)

def plot_flowfield(x, y, u, v):
    u = u.reshape(len(x2_test), len(x1_test))
    v = v.reshape(len(x2_test), len(x1_test))
    fig, ax = plt.subplots(dpi=150)
    ax.streamplot(x1_test, x2_test, u, v, density=1.5, linewidth=0.7, color='w', arrowsize=0.7, broken_streamlines=True)
    img = ax.contourf(x1_test, x2_test, u, levels = np.linspace(-0.2, 1., 100), cmap=sns.color_palette("icefire", as_cmap=True))
    # plt.plot(boundaryNACA4D(0, 0, 12, 1, 100)[:, 0], boundaryNACA4D(0, 0, 12, 1, 100)[:, 1])
    fig.colorbar(img)
    fig.savefig('Lid_u.png')
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_title("$U_x [m/s]$", fontsize=13)
    plt.show()

    fig, ax = plt.subplots(dpi=150)
    ax.streamplot(x1_test, x2_test, u, v, density=1.5, linewidth=0.7, color='w', arrowsize=0.7, broken_streamlines=True)
    img = ax.contourf(x1_test, x2_test, v, levels = np.linspace(-0.5, .3, 100), cmap=sns.color_palette("icefire", as_cmap=True))
    # img = ax.contourf(x1_test, x2_test, v, levels = 100, cmap=sns.color_palette("icefire", as_cmap=True))
    # plt.plot(boundaryNACA4D(0, 0, 12, 1, 100)[:, 0], boundaryNACA4D(0, 0, 12, 1, 100)[:, 1])
    fig.colorbar(img)
    fig.savefig('Lid_v.png')
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_title("$U_y [m/s]$", fontsize=13)
    plt.show()

plot_flowfield(x=X_test[:, 0], y=X_test[:, 1], u=Y_test[:,0], v=Y_test[:,1])

In [23]:
temp = np.concatenate((np.linspace(bbox[0], bbox[1], 101).reshape(-1,1),np.ones((101,1))),axis=1)
temp = torch.tensor(temp)
# y_temp = model.predict(temp.cpu(), operator=output_transform)
y_temp = model.predict(temp.cpu())
print(y_temp[:10])

temp = np.concatenate((np.zeros((101,1)), np.linspace(bbox[0], bbox[1], 101).reshape(-1,1)),axis=1)
temp = torch.tensor(temp)
# y_temp = model.predict(temp.cpu(), operator=output_transform)
y_temp = model.predict(temp.cpu())
print(y_temp[:10])

[[ 0.7906272  -0.11229286 -0.28368402]
 [ 0.85020065 -0.07768084 -0.18926184]
 [ 0.8955242  -0.05165213 -0.11753107]
 [ 0.92940235 -0.0324497  -0.06334058]
 [ 0.95474666 -0.01857872 -0.02203839]
 [ 0.97381234 -0.00875865  0.00989312]
 [ 0.98816824 -0.00193555  0.03493791]
 [ 0.9988873   0.00272027  0.05482296]
 [ 1.0067251   0.00583766  0.07076036]
 [ 1.0122503   0.00788041  0.08362092]]
[[0.01462545 0.00810858 0.1476489 ]
 [0.01481917 0.00794186 0.14764692]
 [0.01500633 0.00777599 0.14764205]
 [0.01518712 0.00761071 0.14763409]
 [0.01536125 0.00744593 0.14762335]
 [0.01552871 0.00728139 0.14760938]
 [0.01568962 0.00711701 0.14759226]
 [0.01584344 0.00695233 0.14757188]
 [0.01599053 0.00678782 0.14754821]
 [0.0161309  0.00662262 0.14752136]]


In [26]:
print(Y_test[:,1].shape)

(10201,)
